In [2]:
!pip install shap
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
!pip install nlp

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
import shap
import transformers
import nlp
import torch
from pathlib import Path
import numpy as np
import scipy as sp

# project_dir = Path.cwd()
# print(project_dir)

from transformers import AutoModelForSequenceClassification
# load a BERT sentiment analysis model
tokenizer = transformers.AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
# cl_path = '/content/drive/MyDrive/finBERT/models/classifier_model/finbert-sentiment'
# model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3).cuda()

# define a prediction function
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x])
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

# build an explainer using a token masker

In [7]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [8]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

df=pd.read_csv("C:\\Users\\fatjo\\Desktop\\dataSpecial.csv", encoding="latin", header=[0])
df

,Description,Semantic
0,Finnish Talentum reports its operating profit ...,positive
1,"Lifetree was founded in 2000 , and its revenue...",positive
2,Nokia also noted the average selling price of ...,positive
3,Calls to the switchboard and directory service...,negative
4,"Earnings per share EPS are seen at EUR 0.56 , ...",positive
5,The growth of net sales has continued favourab...,positive
6,The company slipped to an operating loss of EU...,negative
7,The company 's profit before taxes fell to EUR...,negative
8,Unit costs for flight operations fell by 6.4 p...,negative
9,"Tiimari , the Finnish retailer , reported to h...",positive


In [9]:
desc=df['Description']
print(type(desc))

df2 = desc.to_frame(name="Desc")
print(type(df2))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [10]:
dataset = ds.dataset(pa.Table.from_pandas(df2).to_batches())

### convert to Huggingface dataset
data = Dataset(pa.Table.from_pandas(df2))

In [11]:
explainer = shap.Explainer(f, tokenizer)

# explain the model's predictions on IMDB reviews
shap_values = explainer(data['Desc'][0:10], fixed_context=1)

  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer:  10%|█         | 1/10 [00:00<?, ?it/s]

  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer:  30%|███       | 3/10 [01:43<02:23, 20.52s/it]

  0%|          | 0/248 [00:00<?, ?it/s]

...

Partition explainer:  90%|█████████ | 9/10 [06:38<00:47, 47.45s/it]

  0%|          | 0/182 [00:00<?, ?it/s]

Partition explainer: 100%|██████████| 10/10 [07:00<00:00, 39.59s/it]

  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 11it [08:06, 48.63s/it]                        


In [12]:
shap.plots.text(shap_values[8:9])